In [7]:
import os
import re
import javalang
import subprocess
import pandas as pd
import numpy as np

In [2]:
filename = '../stimuli/pruned_seeds2.csv'
csv = pd.read_csv(filename)

In [3]:
test = csv['function'][2]
test2 = re.sub("\n", "", test)
#test3 = re.sub(" ", "", test2)
test3 = str('import javax.xml.stream.events.Attributes; public class Snippet{'+ test2+ '}')


In [12]:
filedir = os.listdir("wrapped_functions")

['getTargetServiceName_wrapped.java', 'removeRedundantOperations_wrapped.java', 'makeCenter_wrapped.java', 'testGetEvtIDs_wrapped.java', 'getScopePartnerLinks_wrapped.java', 'testCookieGreaterThan_wrapped.java', 'asMap_wrapped.java', 'visitRetStmt_wrapped.java', 'actionLoadNotes_wrapped.java', 'goToRegistration_wrapped.java', 'equals_wrapped.java', 'createChecklistItem_wrapped.java', 'readFromFile_wrapped.java', 'createNewServerProcess_wrapped.java', 'getUserNameFromCookie_wrapped.java', 'removeService_wrapped.java', 'getRodinDBStatus_wrapped.java', 'addToTechnicalComment_wrapped.java', 'removeView_wrapped.java', 'getRemoteObject_wrapped.java', 'getBackCommand22_wrapped.java', 'testSetExample_wrapped.java', 'documentLoadingCompleted_wrapped.java', 'testGetClassification_wrapped.java', 'markStart_wrapped.java', 'countQuery_wrapped.java', 'addDbArgs_wrapped.java', 'resolveValue_wrapped.java', 'genIdents_wrapped.java', 'invalidateSession_wrapped.java', 'getApplicableLaw_wrapped.java', 'to

In [18]:

for file in filedir:
    infile = f"wrapped_functions/{file}"
    outfile = f"{infile[:-5]}.xml"
    # with open(filename, "w") as f:
    #     f.write(snippet)
    subprocess.run(["srcml", infile, "-o", outfile])

I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriterStartDocument : invalid writer!
I/O error : No such file or directory
I/O error : No such file or directory
error : xmlTextWriter

In [6]:
tree = javalang.parse.parse(test3)


In [7]:
tree

CompilationUnit(imports=[Import(path=javax.xml.stream.events.Attributes, static=False, wildcard=False)], package=None, types=[ClassDeclaration(annotations=[], body=[MethodDeclaration(annotations=[], body=[ForStatement(body=BlockStatement(label=None, statements=[StatementExpression(expression=MethodInvocation(arguments=[MethodInvocation(arguments=[MemberReference(member=i, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])], member=getLocalName, postfix_operators=[], prefix_operators=[], qualifier=from, selectors=[], type_arguments=None), MethodInvocation(arguments=[MemberReference(member=i, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[])], member=getValue, postfix_operators=[], prefix_operators=[], qualifier=from, selectors=[], type_arguments=None)], member=setAttribute, postfix_operators=[], prefix_operators=[], qualifier=topMenu, selectors=[], type_arguments=None), label=None), StatementExpression(expression=MethodInvocation(arguments=[MethodInv